In [7]:
import pickle
import numpy as np
from skimage import io
from tqdm.autonotebook import tqdm, trange

from PIL import Image
from pathlib import Path

import time
import torch
import numpy as np
import PIL

from torchvision import datasets, models, transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

# в sklearn не все гладко, чтобы в colab удобно выводить картинки 
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)


In [8]:
!nvidia-smi


Sun Apr 17 18:56:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   42C    P8    14W / 170W |    900MiB / 12288MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
transform = transforms.Compose([
    transforms.Resize((int(244 * 1.25), int(244 * 1.25))),
    transforms.RandomCrop(244),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data = datasets.ImageFolder(
    root='train/dataset',
    transform=transform
)


train_idx, valid_idx = train_test_split(list(range(len(data))), train_size=0.75)
dataset = {
    'train': torch.utils.data.Subset(data, train_idx),
    'val': torch.utils.data.Subset(data, valid_idx)
}

dataloader = {
    'train': torch.utils.data.DataLoader(
        dataset=dataset['train'], batch_size=32, shuffle=True, num_workers=4
    ),
    'val': torch.utils.data.DataLoader(
        dataset=dataset['val'], batch_size=32, shuffle=False, num_workers=4
    ),
}




transform = transforms.Compose([
    transforms.Resize((int(244 * 1.05), int(244 * 1.05))),
    transforms.CenterCrop(244),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_test = datasets.ImageFolder(
    root='testset/',
    transform=transform
)
datasets_sizes = {x: len(dataset[x]) for x in ['train', 'val']}
class_names = np.array(data.classes)
dataloader_test = torch.utils.data.DataLoader(dataset_test, shuffle=False, batch_size=32)

In [18]:
def train_model(model, criterion, optimizer, scheduler, epochs=25):
    '''

    main training code

    '''
    start = time.time()
    
    use_gpu = torch.cuda.is_available()
  
    best_mode_wts = model.state_dict()
    best_accuracy = 0.0

    losses = {'train': [], 'val': []}
    accuracy = {'train': [], 'val': []}

    pbar = trange(epochs, desc='Epoch')

    for epoch in pbar:

        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.eval()

            curr_loss = 0.0
            curr_corrects = 0  

            for data in tqdm(dataloader[phase], leave=False, desc=f'{phase} iter'):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = inputs, labels
        
                if phase == 'train':
                    optimizer.zero_grad()

                if phase == 'val':
                    with torch.no_grad():
                        outputs = model(inputs)
                else:
                    outputs = model(inputs)

                preds = torch.argmax(outputs, -1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                curr_loss += loss.item()
                curr_corrects += int(torch.sum(preds == labels.data))

            epoch_loss = curr_loss / datasets_sizes[phase]
            epoch_accuracy = curr_corrects / datasets_sizes[phase]

            losses[phase].append(epoch_loss)
            accuracy[phase].append(epoch_accuracy)

            pbar.set_description('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_accuracy))

            if phase == 'val' and epoch_accuracy > best_accuracy:
                best_accuracy = epoch_accuracy
                best_model_wts = model.state_dict()
  
    time_elapsed = time.time() - start
    
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60
    ))
    print('Best val Acc: {:.4f}'.format(best_accuracy))

    model.load_state_dict(best_model_wts)
    
    return model, losses, accuracy

def evaluate(model):
    '''

    count accuracy on a test set

    '''

    model.eval()

    curr_correct = 0
    for data in dataloader['val']:
        inputs, labels = data

    if torch.cuda.is_available():
        inputs, labels = inputs.cuda(), labels.cuda()

    output = model(inputs)
    _, preds = torch.max(output, 1)

    curr_correct += int(torch.sum(preds == labels))

    return curr_correct / datasets_sizes['val']

def predict(model, dataloader_test, class_names):
    probs = []
    model.eval()
    with torch.no_grad():
        
        for inputs, y in tqdm(dataloader_test):
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                
            preds = model(inputs).cpu()
            probs.append(preds)
            
    print(f'probs shape before softmax: {len(probs)}')
    probs = nn.functional.softmax(torch.cat(probs), dim=-1).numpy()
    print(f'probs shape after softmax: {probs.shape}')
    probs = np.argmax(probs, axis=1)
    probs = class_names[probs]
    return probs

In [15]:
model_resnet = models.resnet50(pretrained=True)

for param in model_resnet.parameters():
    param.require_grad = False

num_features = model_resnet.fc.in_features
model_resnet.classifier = nn.Linear(num_features, len(data.classes))

model_resnet = model_resnet.cuda()

model_resnet.classifier

Linear(in_features=2048, out_features=42, bias=True)

In [16]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(model_resnet.parameters()), lr=1e-4)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

model_resnet, losses_resnet, accs_resnet = train_model(model_resnet, loss_func, optimizer, exp_lr_scheduler, epochs=12)

Epoch:   0%|          | 0/12 [00:00<?, ?it/s]

train iter:   0%|          | 0/491 [00:00<?, ?it/s]

val iter:   0%|          | 0/164 [00:00<?, ?it/s]

train iter:   0%|          | 0/491 [00:00<?, ?it/s]

val iter:   0%|          | 0/164 [00:00<?, ?it/s]

...

train iter:   0%|          | 0/491 [00:00<?, ?it/s]

val iter:   0%|          | 0/164 [00:00<?, ?it/s]

train iter:   0%|          | 0/491 [00:00<?, ?it/s]

val iter:   0%|          | 0/164 [00:00<?, ?it/s]

Training complete in 31m 48s
Best val Acc: 0.9765


In [33]:
test_filenames = [fn[0].split('\\')[-1] for fn in dataset_test.imgs]
test_filenames

['img0.jpg',
 'img1.jpg',
 'img10.jpg',
 'img100.jpg',
 'img101.jpg',
 'img102.jpg',
 'img103.jpg',
 'img104.jpg',
 'img105.jpg',
 'img106.jpg',
 'img107.jpg',
 'img108.jpg',
 'img109.jpg',
 'img11.jpg',
 'img110.jpg',
 'img111.jpg',
 'img112.jpg',
 'img113.jpg',
 'img114.jpg',
 'img115.jpg',
 'img116.jpg',
 'img117.jpg',
 'img118.jpg',
 'img119.jpg',
 'img12.jpg',
 'img120.jpg',
 'img121.jpg',
 'img122.jpg',
 'img123.jpg',
 'img124.jpg',
 'img125.jpg',
 'img126.jpg',
 'img127.jpg',
 'img128.jpg',
 'img129.jpg',
 'img13.jpg',
 'img130.jpg',
 'img131.jpg',
 'img132.jpg',
 'img133.jpg',
 'img134.jpg',
 'img135.jpg',
 'img136.jpg',
 'img137.jpg',
 'img138.jpg',
 'img139.jpg',
 'img14.jpg',
 'img140.jpg',
 'img141.jpg',
 'img142.jpg',
 'img143.jpg',
 'img144.jpg',
 'img145.jpg',
 'img146.jpg',
 'img147.jpg',
 'img148.jpg',
 'img149.jpg',
 'img15.jpg',
 'img150.jpg',
 'img151.jpg',
 'img152.jpg',
 'img153.jpg',
 'img154.jpg',
 'img155.jpg',
 'img156.jpg',
 'img157.jpg',
 'img158.jpg',
 'img

In [31]:
import pandas as pd
test_filenames = [fn[0].split('\\')[-1] for fn in dataset_test.imgs]
preds_resnet = predict(model_resnet, dataloader_test, class_names)


submission = pd.read_csv('sample_submission.csv')
submission = pd.DataFrame({'id': test_filenames, 'Expected': preds_resnet}).sort_values('id')
submission.to_csv('submission.csv', index=False)

submission

  0%|          | 0/31 [00:00<?, ?it/s]

probs shape before softmax: 31
probs shape after softmax: (991, 1000)


,id,Expected
0,img0.jpg,nelson_muntz
1,img1.jpg,bart_simpson
2,img10.jpg,ned_flanders
3,img100.jpg,chief_wiggum
4,img101.jpg,apu_nahasapeemapetilon
...,...,...
986,img987.jpg,nelson_muntz
987,img988.jpg,ned_flanders
988,img989.jpg,charles_montgomery_burns
989,img99.jpg,chief_wiggum
